
Copyright 2018 Google Inc.

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

> https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Datathon Tutorial

The aim of this tutorial is to get you familiarized with BigQuery to query/filter/aggregate/export data with Python.

## Prerequisites

You should already have had a valid Gmail account registered with the datathon organizers.
  * If you do not have a Gmail account, you can create one at http://www.gmail.com. You need to notify datathon organizers to register your new account for data access.
  * If you have not yet signed the data use agreement (DUA) sent by the organizers, please do so immediately to get access to the MIMIC-III dataset.

## Setup

To be able to run the queries in this tutorial, you need to create a copy of this Colab notebook by clicking "File" > "Save a copy in Drive..." menu.
You can share your copy with your teammates by clicking on the "SHARE" button on the top-right corner of your Colab notebook copy. Everyone with "Edit" permission is able to modify the notebook at the same time, so it is a great way for team collaboration. Before running any cell in this colab, please make sure there is a green check mark before "CONNECTED" on top right corner, if not, please click "CONNECTED" button to connect to a random backend.

Now that you have done the initial setup, let us start playing with the data. First, you need to run some initialization code. You can run the following cell by clicking on the triangle button when you hover over the [ ] space on the top-left corner of the code cell below.

In [0]:
# Import libraries
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.path as path
import tensorflow as tf

# Below imports are used to print out pretty pandas dataframes
from IPython.display import display, HTML

# Imports for accessing Datathon data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

Before running any queries using BigQuery, you need to first authenticate yourself by running the following cell. If you are running it for the first time, it will ask you to follow a link to log in using your Gmail account, and accept the data access requests to your profile. Once this is done, it will generate a string of verification code, which you should paste back to the cell below and press enter.

In [0]:
auth.authenticate_user()

The data-hosting project `physionet-data` has read-only access, as a result, you need to set a default project that you have BigQuery access to. A shared project should be created by the event organizers, and we will be using it throughout this tutorial.

Note that during the datathon, all participants will be divided into teams and a Google Cloud project will be created for each team specifically. That project would be the preferred project to use. For now we'll stick with the shared project for the purpose of the tutorial.

After datathon is finished, the shared project may either lock down access or be deleted, it's still possible to run queries from a project you own personally as long as you have access to the dataset hosting project.

Change the variable project_id below to list the project you are using.

In [0]:
project_id='datathon-tarragona-2018'
os.environ["GOOGLE_CLOUD_PROJECT"]=project_id

Let's define a few methods to wrap BigQuery operations, so that we don't have to write the configurations again and again.

In [0]:
# Read data from BigQuery into pandas dataframes.
def run_query(query):
  return pd.io.gbq.read_gbq(query, project_id=project_id, verbose=False, configuration={'query':{'useLegacySql': False}})

OK, that's it for setup, now let's get our hands on the MIMIC demo data!

# SQL query

In [0]:
df1 = run_query('''
SELECT int.patientunitstayid, apache.apacheadmissiondx, icu.unittype, icu.apache_iv, labfirst.aniongap_min, labfirst.aniongap_max, labfirst.albumin_min, labfirst.albumin_max, labfirst.bands_min, labfirst.bands_max, labfirst.bicarbonate_min, labfirst.bicarbonate_max, labfirst.hco3_min, labfirst.hco3_max, labfirst.bilirubin_min, labfirst.bilirubin_max, labfirst.creatinine_min, labfirst.creatinine_max, labfirst.chloride_min, labfirst.chloride_max, labfirst.glucose_min, labfirst.glucose_max, labfirst.hematocrit_min, labfirst.hematocrit_max, labfirst.hemoglobin_min, labfirst.hemoglobin_max, labfirst.lactate_min, labfirst.lactate_max, labfirst.potassium_min, labfirst.potassium_max, labfirst.sodium_min, labfirst.sodium_max, labfirst.bun_min, labfirst.bun_max, pbg.fio2, pbg.pao2,
       pinf.dopamine, pinf.dobutamine, pinf.norepinephrine, puo.outputtotal, puo.urineoutput ,pvital.heartrate, pvital.respiratoryrate, pvital.spo2, pvital.nibp_systolic, pvital.nibp_diastolic, pvital.nibp_mean, pvital.temperature, pvital.temperaturelocation, pvital.ibp_systolic, pvital.ibp_diastolic, pvital.ibp_mean, dem.age, dem.gender, dem.icu_los_hour, dem.hosp_mortality
FROM `datathon-tarragona-2018.team7.intersect_aki_dialysis` int, `physionet-data.eicu_crd_derived.apache_groups` apache, `physionet-data.eicu_crd_derived.icustay_detail` icu, `physionet-data.eicu_crd_derived.labsfirstday` labfirst, `physionet-data.eicu_crd_derived.pivoted_bg` pbg, `physionet-data.eicu_crd_derived.pivoted_infusion` pinf, `physionet-data.eicu_crd_derived.pivoted_uo` puo, `physionet-data.eicu_crd_derived.pivoted_vital` pvital, `physionet-data.eicu_crd_derived.basic_demographics` dem
WHERE int.patientunitstayid=apache.patientunitstayid and int.patientunitstayid=icu.patientunitstayid and int.patientunitstayid=labfirst.patientunitstayid and int.patientunitstayid=pbg.patientunitstayid and int.patientunitstayid=pinf.patientunitstayid and int.patientunitstayid=puo.patientunitstayid and int.patientunitstayid=pvital.patientunitstayid and int.patientunitstayid=dem.patientunitstayid
''')
df1.head()

In [6]:
dataset = run_query('''
SELECT * 
FROM `datathon-tarragona-2018.team7.int3_def`
''')
dataset.head()

,patientunitstayid,outputtotal,urineoutput,heartrate,respiratoryrate,spo2,nibp_systolic,nibp_diastolic,nibp_mean,temperature,temperaturelocation,ibp_systolic,ibp_diastolic,ibp_mean,age,gender,icu_los_hours,hosp_mortality
0,3051522,1.0,1.0,73.0,20.0,89.0,NaN,NaN,NaN,NaN,None,30.0,27.0,28.0,33,1,258.0,1
1,3051522,200.0,200.0,73.0,20.0,89.0,NaN,NaN,NaN,NaN,None,30.0,27.0,28.0,33,1,258.0,1
2,3015427,559.0,18.0,116.0,NaN,89.0,150.0,79.0,94.0,NaN,None,183.0,85.0,125.0,69,1,310.0,0
3,3015427,512.0,0.0,116.0,NaN,89.0,150.0,79.0,94.0,NaN,None,183.0,85.0,125.0,69,1,310.0,0
4,3341064,210.0,210.0,80.0,26.0,71.0,NaN,NaN,NaN,NaN,None,NaN,110.0,176.0,82,2,1121.0,0


In [0]:
dataset.head()

# Sección pruebas Albert

In [0]:
#Setup
# Import libraries
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.path as path
import tensorflow as tf

# Below imports are used to print out pretty pandas dataframes
from IPython.display import display, HTML

# Imports for accessing Datathon data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

auth.authenticate_user()

project_id='datathon-tarragona-2018'
os.environ["GOOGLE_CLOUD_PROJECT"]=project_id

# Read data from BigQuery into pandas dataframes.
def run_query(query):
  return pd.io.gbq.read_gbq(query, project_id=project_id, verbose=False, configuration={'query':{'useLegacySql': False}})

In [0]:
df1 = run_query('''
  SELECT
  *
  FROM `datathon-tarragona-2018.team7.intersect_aki_dialysis`
''')
df1.head()

In [7]:
df2 = run_query('''
  SELECT
  *
  FROM `datathon-tarragona-2018.team7.outer_aki_notdialysis`
''')
df2.head()

df3 = run_query('''
  SELECT patientUnitStayID FROM `physionet-data.eicu_crd.patient`
''')

df3.describe()

df3.head()
#count(df3[''].unique())

,patientUnitStayID
0,1137569
1,3036927
2,3058863
3,3072720
4,3075429


In [9]:
len(df3['patientUnitStayID'].unique())

200859

# Graphical Interface

In [10]:
Age = 23 #@param {type:"slider", min:10, max:99, step:1}
Gender = "Female" #@param ["Male", "Female"]
Creatinine = 65 #@param {type:"number"}

if Age < 50:
  print("The patient is healthy.")
else:
  print("Hay que amputar")

The patient is healthy.


# Modelos

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
#dataset = pd.read_csv('/Users/uxuelazkanodobao/Documents/DATATHON/iris.csv')

# Any results you write to the current directory are saved as output.

In [8]:
dataset.columns

Index([u'patientunitstayid', u'outputtotal', u'urineoutput', u'heartrate',
       u'respiratoryrate', u'spo2', u'nibp_systolic', u'nibp_diastolic',
       u'nibp_mean', u'temperature', u'temperaturelocation', u'ibp_systolic',
       u'ibp_diastolic', u'ibp_mean', u'age', u'gender', u'icu_los_hours',
       u'hosp_mortality'],
      dtype='object')

In [9]:
dataset['hosp_mortality'].describe()

count     99948
unique        2
top           1
freq      50264
Name: hosp_mortality, dtype: int64

In [0]:
dataset = dataset.drop(columns= 'temperaturelocation')

In [0]:
dataset = dataset[(dataset['age']!='> 89')]


In [0]:
dataset["age"] = pd.to_numeric(dataset["age"])


In [50]:
dataset.columns

Index([u'patientunitstayid', u'outputtotal', u'urineoutput', u'heartrate',
       u'respiratoryrate', u'spo2', u'nibp_systolic', u'nibp_diastolic',
       u'nibp_mean', u'temperature', u'ibp_systolic', u'ibp_diastolic',
       u'ibp_mean', u'age', u'gender', u'icu_los_hours', u'hosp_mortality'],
      dtype='object')

In [0]:
dataset["hosp_mortality"] = pd.to_numeric(dataset["hosp_mortality"])


In [0]:
x = dataset.iloc[:,:16].values
y= dataset.iloc[:,-1].values

In [0]:
# Split the data into Training and Testing set
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=0)

In [0]:
dataset = dataset.dropna()

In [0]:
# Feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

In [20]:
y

array([0., 0., 0., ..., 1., 1., 1.])

In [21]:
#Fitting logistic regression to the training set
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [0]:
# Predicting the Test set results
y_pred = classifier.predict(x_test)

In [0]:
# Making the confusion matrix 
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)

In [24]:
# Making the confusion matrix 
from sklearn import metrics
print (metrics.accuracy_score(y_test,classifier.predict(x_test)))

0.8293591654247392


In [25]:
#Improve the technique of logistic regression
predict_prob_df=pd.DataFrame(classifier.predict_proba(x_test))
predict_prob_df.head(5)

,0,1
0,0.090157,0.909843
1,0.823351,0.176649
2,0.070416,0.929584
3,0.575136,0.424864
4,0.818162,0.181838


# SVM-Support Vector Machine for Classification


In [40]:
# Support Vector Machines
from sklearn.svm import SVC
svc = SVC()
svc.fit(x_train, y_train)
y_pred = svc.predict(x_test)
acc_svc = round(svc.score(x_test, y_test) * 100, 2)
acc_svc

95.98

In [41]:
# Linear SVC
from sklearn.svm import LinearSVC
linear_svc = LinearSVC()
linear_svc.fit(x_train, y_train)
y_pred = linear_svc.predict(x_test)
acc_linear_svc = round(linear_svc.score(x_test, y_test) * 100, 2)
acc_linear_svc

83.08

# Logistic regression

In [42]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(x_train, y_train)
y_pred = logreg.predict(x_test)
acc_log = round(logreg.score(x_test, y_test) * 100, 2)
acc_log

82.94

# KNN - K Nearest Neighbour Prediction


In [43]:
#KNN - K-Nearest Neighbour
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(x_train, y_train)
y_pred = knn.predict(x_test)
acc_knn = round(knn.score(x_test, y_test) * 100, 2)
acc_knn

99.11

# Gaussian Naive Bayes

In [44]:
# Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB
gaussian = GaussianNB()
gaussian.fit(x_train, y_train)
y_pred = gaussian.predict(x_test)
acc_gaussian = round(gaussian.score(x_test, y_test) * 100, 2)
acc_gaussian

69.97

# Perceptron Model/Algorithm analysis

In [38]:
# Perceptron
from sklearn.linear_model import Perceptron
perceptron = Perceptron()
perceptron.fit(x_train, y_train)
y_pred = perceptron.predict(x_test)
acc_perceptron = round(perceptron.score(x_test, y_test) * 100, 2)
acc_perceptron

75.48

# Stochastic Gradient Descent

In [45]:
# Stochastic Gradient Descent
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
acc_sgd = round(clf.score(x_test, y_test) * 100, 2)
acc_sgd

81.15

# Decision Tree Algorithm 

In [33]:
# Decision Tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(x_train, y_train)
y_pred = decision_tree.predict(x_test)
acc_decision_tree = round(decision_tree.score(x_train, y_train) * 100, 2)
acc_decision_tree

NameError: ignored

# Random Forest

In [46]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(x_train, y_train)
y_pred = random_forest.predict(x_test)
random_forest.score(x_train, y_train)
acc_random_forest = round(random_forest.score(x_test, y_test) * 100, 2)
acc_random_forest

98.88

# Model evaluation 



In [48]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              ],
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_linear_svc]})
models.sort_values(by='Score', ascending=False)

,Model,Score
1,KNN,99.11
3,Random Forest,98.88
0,Support Vector Machines,95.98
7,Linear SVC,83.08
2,Logistic Regression,82.94
6,Stochastic Gradient Decent,81.15
5,Perceptron,75.48
4,Naive Bayes,69.97


In [0]:
#Build a model to predict y using x1,x2 and x3. Note down R-Square and Adj R-Square values 
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(x_train, y_train)
predictions = lr.predict(x_test)

In [52]:
import statsmodels.formula.api as sm
model = sm.ols(formula='hosp_mortality ~ patientunitstayid+outputtotal+urineoutput+heartrate+respiratoryrate+spo2+nibp_systolic+nibp_diastolic+nibp_mean+temperature+ibp_systolic+ibp_diastolic+ibp_mean+age+gender+icu_los_hours', data=dataset)
fitted1 = model.fit()
fitted1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         hosp_mortality   R-squared:                       0.454
Model:                            OLS   Adj. R-squared:                  0.453
Method:                 Least Squares   F-statistic:                     278.3
Date:                Sun, 11 Nov 2018   Prob (F-statistic):               0.00
Time:                        15:57:03   Log-Likelihood:                -2265.1
No. Observations:                5367   AIC:                             4564.
Df Residuals:                    5350   BIC:                             4676.
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             3.0077      0.279     10.798      0.000       2.462       3.554
patientunitstayid  7.866e-08   6.08e-09     12.935      0.000    6.67e-08    9.06e-08
outputtotal        9.953e-06   9.27e-06      1.074      0.283   -8.21e-06    2.81e-05
urineoutput          -0.0003   3.66e-05     -7.038      0.000      -0.000      -0.000
heartrate             0.0073      0.000     20.728      0.000       0.007       0.008
respiratoryrate      -0.0021      0.001     -2.555      0.011      -0.004      -0.000
spo2                 -0.0043      0.001     -2.924      0.003      -0.007      -0.001
nibp_systolic        -0.0105      0.001    -20.467      0.000      -0.012      -0.010
nibp_diastolic       -0.0198      0.001    -15.000      0.000      -0.022      -0.017
nibp_mean             0.0244      0.002     15.753      0.000       0.021       0.027
temperature          -0.0685      0.006    -11.021      0.000      -0.081      -0.056
ibp_systolic         -0.0034      0.000     -8.575      0.000      -0.004      -0.003
ibp_diastolic        -0.0072      0.001     -8.143      0.000      -0.009      -0.005
ibp_mean              0.0097      0.001      8.390      0.000       0.007       0.012
age                   0.0041      0.000      9.296      0.000       0.003       0.005
gender                0.0993      0.013      7.574      0.000       0.074       0.125
icu_los_hours        -0.0002   2.34e-05     -9.249      0.000      -0.000      -0.000
==============================================================================
Omnibus:                       20.873   Durbin-Watson:                   0.234
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               23.564
Skew:                           0.100   Prob(JB):                     7.64e-06
Kurtosis:                       3.256   Cond. No.                     1.44e+08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.44e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""